In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pygam import GAM, ExpectileGAM, s, l, f
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
from plotnine import *
import pytimetk as tk
import textwrap

In [2]:
df = pd.read_excel('Mexico_Sales_Data_20180101_20240630.xlsx', sheet_name='Data')

df = df[df['UnitPrice'] >= 0]
df = df[df['UnitPrice'] != 315.93]



df['UnitPrice'] = df['UnitPrice'].astype(float)
df['Quantity'] = df['Quantity'].astype(float)
df['UnitCost'] = df['UnitCost'].astype(float)


df['Revenue'] = df['UnitPrice'] * df['Quantity']

label_encoder = LabelEncoder()
df['LongItem_encoded'] = label_encoder.fit_transform(df['LongItem'])

df['UnitPrice'] = pd.to_numeric(df['UnitPrice'], errors='coerce')
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Revenue'] = pd.to_numeric(df['Revenue'], errors='coerce')
df['UnitCost'] = pd.to_numeric(df['UnitCost'], errors='coerce')



df = df.dropna(subset=['UnitPrice', 'Quantity'])

df

,OrderDate_Year,OrderDate_Month,LongItem,UnitListPrice,UnitPrice,UnitCost,Quantity,Revenue,LongItem_encoded
0,2019,4,MX2485,3.0203,1.26,1.144,3000.0,3780.0,1
1,2019,12,MX2485,3.0203,1.26,1.144,6000.0,7560.0,1
2,2020,1,MX2485,3.0203,1.26,1.160,2000.0,2520.0,1
3,2019,7,MX2485,3.0203,1.27,1.144,4000.0,5080.0,1
4,2019,9,MX2485,3.0203,1.27,1.144,4000.0,5080.0,1
...,...,...,...,...,...,...,...,...,...
5967,2024,2,MX89172,260.0000,124.80,62.140,21906.0,2733868.8,3
5968,2024,3,MX89172,260.0000,124.80,62.140,29364.0,3664627.2,3
5969,2024,4,MX89172,260.0000,124.80,62.140,27390.0,3418272.0,3
5970,2024,5,MX89172,260.0000,124.80,62.140,23880.0,2980224.0,3


## Summary Stats

In [3]:
def format_currency(value):
    return "${:,.2f}".format(value)

def format_quantity(value):
    return "{:,.2f}".format(value)

summary_stats = df.groupby('LongItem').agg({
    'Quantity': ['sum', 'mean', 'median', 'std', 'min', 'max'],
    'UnitPrice': ['mean', 'median', 'std', 'min', 'max'],
    'Revenue': ['sum', 'mean', 'median', 'std', 'min', 'max']
}).reset_index()

summary_stats.columns = ['_'.join(col).strip() for col in summary_stats.columns.values]  # Flattening the MultiIndex columns
summary_stats.rename(columns={'LongItem_': 'LongItem'}, inplace=True)

currency_columns = ['UnitPrice_mean','UnitPrice_min', 'UnitPrice_max', 'Revenue_median',
                    'Revenue_sum', 'Revenue_mean','Revenue_min', 'Revenue_max', 'UnitPrice_median']

quantity_columns = ['Quantity_sum','Quantity_mean', 'Quantity_std', 'Quantity_min',
                    'Quantity_max', 'UnitPrice_std','Revenue_std', 'Quantity_median']

# Apply the formatting function only to the currency_columns
summary_stats[currency_columns] = summary_stats[currency_columns].applymap(format_currency)

summary_stats[quantity_columns] = summary_stats[quantity_columns].applymap(format_quantity)

summary_stats

,LongItem,Quantity_sum,Quantity_mean,Quantity_median,Quantity_std,Quantity_min,Quantity_max,UnitPrice_mean,UnitPrice_median,UnitPrice_std,UnitPrice_min,UnitPrice_max,Revenue_sum,Revenue_mean,Revenue_median,Revenue_std,Revenue_min,Revenue_max
0,MX1,"1,558,453.00",488.54,142.00,"1,114.66",-924.00,"21,456.00",$35.15,$34.68,6.26,$21.85,$57.93,"$50,364,406.23","$15,788.22","$4,659.84","34,850.48","$-25,696.44","$692,385.12"
1,MX2485,"2,841,338.00","9,195.27","4,000.00","14,970.31","-6,000.00","113,000.00",$1.88,$1.85,0.35,$1.26,$3.40,"$5,256,475.39","$17,011.25","$7,350.00","28,241.98","$-13,260.00","$206,790.00"
2,MX3399,"2,262,176.00",977.18,156.00,"2,088.99","-1,872.00","22,728.00",$30.90,$30.28,5.42,$16.82,$52.26,"$64,186,822.08","$27,726.49","$5,013.90","59,526.99","$-50,132.16","$711,590.16"
3,MX89172,"1,396,413.00","8,838.06",875.00,"11,894.82","-4,476.00","47,604.00",$97.27,$95.08,18.06,$66.44,$124.80,"$158,320,030.64","$1,002,025.51","$71,162.35","1,414,775.68","$-396,528.84","$5,940,979.20"


## Quantity Distribution

In [4]:
fig = px.box(df, x='LongItem', y='Quantity', title='Quantity Distribution per LongItem')
fig.show()

Min (Minimum): The smallest data point within the dataset, excluding outliers. It is represented by the lower whisker of the plot.

Q1 (First Quartile): This is the median of the lower half of the dataset. 25% of the data points are below this value. It forms the bottom edge of the box.

Median (Q2 or Second Quartile): This is the middle value of the dataset, dividing it into two equal parts. 50% of the data points are below this value. It is represented by a line inside the box.

Q3 (Third Quartile): This is the median of the upper half of the dataset. 75% of the data points are below this value. It forms the top edge of the box.

Max (Maximum): The largest data point within the dataset, excluding outliers. It is represented by the upper whisker of the plot.

Lower Fence: This is calculated as 
𝑄
1
−
1.5
×
IQR
Q1−1.5×IQR where IQR (Interquartile Range) is 
𝑄
3
−
𝑄
1
Q3−Q1. Data points below this value are considered outliers.

Upper Fence: This is calculated as 
𝑄
3
+
1.5
×
IQR
Q3+1.5×IQR. Data points above this value are considered outliers.

Outliers: These are data points that fall below the lower fence or above the upper fence. They are usually represented by individual points beyond the whiskers.

## Unit Price Distribution

In [5]:
fig = px.box(df, x='LongItem', y='UnitPrice', title='UnitPrice Distribution per LongItem')
fig.show()

Min (Minimum): The smallest data point within the dataset, excluding outliers. It is represented by the lower whisker of the plot.

Q1 (First Quartile): This is the median of the lower half of the dataset. 25% of the data points are below this value. It forms the bottom edge of the box.

Median (Q2 or Second Quartile): This is the middle value of the dataset, dividing it into two equal parts. 50% of the data points are below this value. It is represented by a line inside the box.

Q3 (Third Quartile): This is the median of the upper half of the dataset. 75% of the data points are below this value. It forms the top edge of the box.

Max (Maximum): The largest data point within the dataset, excluding outliers. It is represented by the upper whisker of the plot.

Lower Fence: This is calculated as 
𝑄
1
−
1.5
×
IQR
Q1−1.5×IQR where IQR (Interquartile Range) is 
𝑄
3
−
𝑄
1
Q3−Q1. Data points below this value are considered outliers.

Upper Fence: This is calculated as 
𝑄
3
+
1.5
×
IQR
Q3+1.5×IQR. Data points above this value are considered outliers.

Outliers: These are data points that fall below the lower fence or above the upper fence. They are usually represented by individual points beyond the whiskers.

## Revenue Distribution

In [6]:
fig = px.box(df, x='LongItem', y='Revenue', title='Revenue Distribution per LongItem')
fig.show()

Min (Minimum): The smallest data point within the dataset, excluding outliers. It is represented by the lower whisker of the plot.

Q1 (First Quartile): This is the median of the lower half of the dataset. 25% of the data points are below this value. It forms the bottom edge of the box.

Median (Q2 or Second Quartile): This is the middle value of the dataset, dividing it into two equal parts. 50% of the data points are below this value. It is represented by a line inside the box.

Q3 (Third Quartile): This is the median of the upper half of the dataset. 75% of the data points are below this value. It forms the top edge of the box.

Max (Maximum): The largest data point within the dataset, excluding outliers. It is represented by the upper whisker of the plot.

Lower Fence: This is calculated as 
𝑄
1
−
1.5
×
IQR
Q1−1.5×IQR where IQR (Interquartile Range) is 
𝑄
3
−
𝑄
1
Q3−Q1. Data points below this value are considered outliers.

Upper Fence: This is calculated as 
𝑄
3
+
1.5
×
IQR
Q3+1.5×IQR. Data points above this value are considered outliers.

Outliers: These are data points that fall below the lower fence or above the upper fence. They are usually represented by individual points beyond the whiskers.

## Quantity vs Unit Price

In [7]:
fig = px.scatter(df, x='UnitPrice', y='Quantity', color='LongItem', title='Quantity vs UnitPrice')
fig.show()

## Revenue vs Unit Price

In [8]:
fig = px.scatter(df, x='UnitPrice', y='Revenue', color='LongItem', title='Revenue vs UnitPrice')
fig.show()

## Monthly Revenue Trend

In [9]:
df['OrderDate'] = pd.to_datetime(df['OrderDate_Year'].astype(str) + '-' + df['OrderDate_Month'].astype(str) + '-01')
monthly_revenue = df.groupby(['OrderDate', 'LongItem']).agg({'Revenue': 'sum'}).reset_index()

fig = px.line(monthly_revenue, x='OrderDate', y='Revenue', color='LongItem', title='Monthly Revenue Trend per LongItem')
fig.show()


## Correlation between Quantity, Unit Price and Revenue

In [10]:
df_new = df.copy()

unique_items = df['LongItem'].unique()

# Create a dictionary to hold correlation matrices
correlation_matrices = {}

# Loop through each unique 'LongItem'
for item in unique_items:
    # Filter the DataFrame for the current 'LongItem'
    filtered_df = df_new[df_new['LongItem'] == item]
    
    # Compute the correlation matrix for the filtered DataFrame
    correlation_matrix = filtered_df[['Quantity', 'UnitPrice', 'Revenue']].corr()
    
    # Store the correlation matrix in the dictionary
    correlation_matrices[item] = correlation_matrix

# Display each correlation matrix as a table
for item, matrix in correlation_matrices.items():
    print(f"Correlation matrix for LongItem = '{item}':")
    display(matrix)
    print("\n" + "="*50 + "\n")

Correlation matrix for LongItem = 'MX2485':


,Quantity,UnitPrice,Revenue
Quantity,1.000000,-0.050623,0.990966
UnitPrice,-0.050623,1.000000,0.020778
Revenue,0.990966,0.020778,1.000000




Correlation matrix for LongItem = 'MX3399':


,Quantity,UnitPrice,Revenue
Quantity,1.00000,-0.217620,0.990930
UnitPrice,-0.21762,1.000000,-0.175083
Revenue,0.99093,-0.175083,1.000000




Correlation matrix for LongItem = 'MX1':


,Quantity,UnitPrice,Revenue
Quantity,1.000000,-0.198789,0.989121
UnitPrice,-0.198789,1.000000,-0.147610
Revenue,0.989121,-0.147610,1.000000




Correlation matrix for LongItem = 'MX89172':


,Quantity,UnitPrice,Revenue
Quantity,1.000000,0.666800,0.991982
UnitPrice,0.666800,1.000000,0.701691
Revenue,0.991982,0.701691,1.000000


## GAM Model Initialization and Fitting

In [11]:
pd.options.display.float_format = '{:.2f}'.format

df_agg = df.groupby(['LongItem', 'UnitPrice']).agg({
    'Quantity': 'sum',
    'Revenue': 'sum'
}).reset_index()

# Creating a Revenue column again after aggregation
df_agg['Revenue'] = df_agg['UnitPrice'] * df_agg['Quantity']

In [12]:
gams = {}
for item in df_agg['LongItem'].unique():
    subset = df_agg[df_agg['LongItem'] == item]
    X = subset[['UnitPrice']]
    y = subset['Revenue']
    gam = GAM(s(0)).fit(X, y)
    gams[item] = gam

df['Predicted_Revenue'] = np.nan
for item in gams:
    mask = df_agg['LongItem'] == item
    df_agg.loc[mask, 'Predicted_Revenue'] = gams[item].predict(df_agg[mask][['UnitPrice']])

## Product Sales: Price vs Revenue Analysis

In [13]:
fig = px.scatter(
    df_agg,
    x='UnitPrice',
    y='Revenue',
    color='LongItem',
    opacity=0.6,
    trendline="lowess",
    trendline_color_override="blue",
    template="none",
    title='Product Sales: Price vs Revenue Analysis',
    width=800,
    height=600,
).update_traces(
    marker=dict(size=7),
    hoverlabel=dict(font=dict(size=10)),
).update_layout(
    legend_title_text='LongItem',
    title_font=dict(size=16),
    legend_font=dict(size=10),
).update_xaxes(
    title_text='UnitPrice',
    title_font=dict(size=10),
    tickfont=dict(size=10),
).update_yaxes(
    title_text='Revenue',
    title_font=dict(size=10),
    tickfont=dict(size=10),
)

fig.show()


## Python Predicted Revenue: Price vs Revenue Analysis

In [14]:
fig = px.scatter(
    df_agg,
    x='UnitPrice',
    y='Predicted_Revenue',
    color='LongItem',
    opacity=0.6,
    template="none",
    title='GAM Predicted Revenue: Price vs Revenue Analysis',
    width=800,
    height=600,
).update_traces(
    marker=dict(size=7),
    hoverlabel=dict(font=dict(size=10)),
).update_layout(
    legend_title_text='LongItem',
    title_font=dict(size=16),
    legend_font=dict(size=10),
).update_xaxes(
    title_text='UnitPrice',
    title_font=dict(size=10),
    tickfont=dict(size=10),
).update_yaxes(
    title_text='Predicted Revenue',
    title_font=dict(size=10),
    tickfont=dict(size=10),
)

fig.show()

## Optimal Price for Maximized Revenues Algorithm

In [15]:
optimal_prices = {}
max_revenues = {}
for item in df_agg['LongItem'].unique():
    subset = df_agg[df_agg['LongItem'] == item]
    gam = gams[item]
    prices = np.linspace(subset['UnitPrice'].min(), subset['UnitPrice'].max(), 100)
    predicted_revenues = gam.predict(prices.reshape(-1, 1))
    optimal_price = prices[np.argmax(predicted_revenues)]
    max_revenue = max(predicted_revenues)
    optimal_prices[item] = optimal_price
    max_revenues[item] = max_revenue
    print(f'Optimal price for {item}: ${optimal_price:,.2f}, Maximized Revenue: ${max_revenue:,.2f}')


Optimal price for MX1: $34.97, Maximized Revenue: $109,560.52
Optimal price for MX2485: $1.97, Maximized Revenue: $132,994.38
Optimal price for MX3399: $31.49, Maximized Revenue: $167,817.32
Optimal price for MX89172: $124.80, Maximized Revenue: $69,795,644.49


In [16]:
# Lower and upper bound graph - Incorporate cost as well


In [17]:
df = df[df['UnitPrice'] <= 500]

columns_to_keep = ['LongItem', 'UnitPrice', 'Quantity', 'Revenue']

data = df[columns_to_keep]

data

,LongItem,UnitPrice,Quantity,Revenue
0,MX2485,1.26,3000.00,3780.00
1,MX2485,1.26,6000.00,7560.00
2,MX2485,1.26,2000.00,2520.00
3,MX2485,1.27,4000.00,5080.00
4,MX2485,1.27,4000.00,5080.00
...,...,...,...,...
5967,MX89172,124.80,21906.00,2733868.80
5968,MX89172,124.80,29364.00,3664627.20
5969,MX89172,124.80,27390.00,3418272.00
5970,MX89172,124.80,23880.00,2980224.00


In [18]:
fig = px.scatter(
    data,
    x='UnitPrice',
    y='Quantity',
    color='LongItem',
    opacity=0.6,
    trendline="lowess",
    trendline_color_override="blue",
    template="none",
    title='Product Sales: Price vs Quantity Analysis',
    width=800,
    height=600,
).update_traces(
    marker=dict(size=7),
    hoverlabel=dict(font=dict(size=10)),
).update_layout(
    legend_title_text='LongItem',
    title_font=dict(size=16),
    legend_font=dict(size=10),
).update_xaxes(
    title_text='UnitPrice',
    title_font=dict(size=10),
    tickfont=dict(size=10),
).update_yaxes(
    title_text='Quantity',
    title_font=dict(size=10),
    tickfont=dict(size=10),
)


fig

In [19]:
fig = px.scatter(
    data,
    x='UnitPrice',
    y='Quantity',
    color='LongItem',
    opacity=0.6,
    trendline="lowess",
    trendline_color_override="blue",
    template="none",
    title='Product Sales: Price vs Quantity Analysis',
    width=800,
    height=600,
).update_traces(
    marker=dict(size=7),
    hoverlabel=dict(font=dict(size=10)),
).update_layout(
    legend_title_text='LongItem',
    title_font=dict(size=16),
    legend_font=dict(size=10),
).update_xaxes(
    title_text='UnitPrice',
    title_font=dict(size=10),
    tickfont=dict(size=10),
).update_yaxes(
    title_text='Quantity',
    title_font=dict(size=10),
    tickfont=dict(size=10),
)


fig